# 合并表格

In [1]:
import numpy as np
import pandas as pd

# ### 读取
# trd_dalyr0=pd.read_excel("stockData/furtherInformation/日个股回报率文件185043337(仅供中央财经大学使用)/TRD_Dalyr.xlsx")
# trd_dalyr0=trd_dalyr0.drop([0,1],axis=0)
# trd_dalyr1=pd.read_excel("stockData/furtherInformation/日个股回报率文件185043337(仅供中央财经大学使用)/TRD_Dalyr1.xlsx")
# trd_dalyr1=trd_dalyr1.drop([0,1],axis=0)
# trd_dalyr2=pd.read_excel("stockData/furtherInformation/日个股回报率文件185043337(仅供中央财经大学使用)/TRD_Dalyr2.xlsx")
# trd_dalyr2=trd_dalyr2.drop([0,1],axis=0)
# trd_dalyr3=pd.read_excel("stockData/furtherInformation/日个股回报率文件185043337(仅供中央财经大学使用)/TRD_Dalyr3.xlsx")
# trd_dalyr3=trd_dalyr3.drop([0,1],axis=0)
# trd_dalyr4=pd.read_excel("stockData/furtherInformation/日个股回报率文件185043337(仅供中央财经大学使用)/TRD_Dalyr4.xlsx")
# trd_dalyr4=trd_dalyr4.drop([0,1],axis=0)
# diaryData=[trd_dalyr0,trd_dalyr1,trd_dalyr2,trd_dalyr3,trd_dalyr4]
# finalDF=pd.concat(diaryData)
# finalDF

In [2]:
# companyData=pd.read_excel("stockData/furtherInformation/公司文件184301535(仅供中央财经大学使用)/TRD_Co.xlsx")
# companyData=companyData.drop([0,1],axis=0)
# companyData

In [3]:
# companyAssetData=pd.read_excel("stockData/furtherInformation/资产负债表(联表查询)181028530/FS_Combas(Merge Query).xlsx")
# companyAssetData=companyAssetData.drop([0,1],axis=0)
# companyAssetData

In [4]:
# ### 去除b部分
# companyAssetData=companyAssetData[companyAssetData["FS_Combas.Typrep"]!="B"]
# companyAssetData=companyAssetData.reindex()
# ### 去除12-31，因为与1-1有重复
# companyAssetData=companyAssetData[companyAssetData['FS_Combas.Accper'].str.contains("01-01|03-31|06-30|09-30")]
# companyAssetData['FS_Combas.Accper']=pd.to_datetime(companyAssetData['FS_Combas.Accper'], format="%Y-%m-%d")
# 
# companyAssetData=companyAssetData.rename(columns={"FS_Combas.Stkcd":"Stkcd","FS_Combas.Accper":"Trddt_quarter"})
# companyAssetData

In [5]:
# ### todo：开始合并 日个股回报与公司信息
# df=pd.merge(finalDF,companyData,on='Stkcd')
# df['Trddt']=pd.to_datetime(df["Trddt"], format="%Y-%m-%d")
# df

In [6]:
# ###转换时间为季度
# def dateToQuarter(date):
#         # print("time: ",date)
#         quarter_date={1:"01-01",2:"03-31",3:"06-30",4:"09-30"}
#         year=date.astype('datetime64[Y]').astype(int) + 1970
#         month = date.astype('datetime64[M]').astype(int) % 12 + 1
#         the_quarter = (month - 1) // 3 + 1                  #计算季度
#         quarter_name = str(year) + '年' + str(the_quarter) + '季度'
#         quarter_name_short = str(year) + 'Q' + str(the_quarter)
#         # print("quarter time: ",quarter_name)
#         return str(year)+"-"+quarter_date[the_quarter]
# 
# # dateToQuarter(numpy.datetime64("2020-04-30"))

In [7]:
# df["Trddt_quarter"]=[dateToQuarter(i) for i in df['Trddt'].values]
# df["Trddt_quarter"]=pd.to_datetime(df["Trddt_quarter"], format="%Y-%m-%d")
# df

In [8]:
# ###todo：根据季度与股票代码合并之前的总表与资产负债表
# allData=pd.merge(df,companyAssetData,on=["Stkcd",'Trddt_quarter'],how='outer')
# # allData['Stkcd']=allData['Stkcd'].astype(str)
# allData

In [9]:
# allData.to_csv("stockData/furtherInformation/AllData.csv")

# 数据去重，并且删除部分缺失值

In [10]:
allData=pd.read_csv("stockData/furtherInformation/AllData.csv",index_col=0,dtype=object)
allData

,Stkcd,Trddt,Opnprc,Hiprc,Loprc,Clsprc,ChangeRatio,Stknme,Listdt,Cuntrycd,...,FS_Comins.B001200000,FS_Comins.B001209000,FS_Comins.B001210000,FS_Comins.B001211000,FS_Comins.B002000000,FS_Comins.ShortName,FS_Comscfd.C001000000,FS_Comscfd.C002000000,FS_Comscfd.C003000000,FS_Comscfd.ShortName
0,000001,2019-01-02,9.39,9.42,9.16,9.19,-0.020256,平安银行,1991-04-03,10,...,36540000000,NaN,NaN,NaN,24818000000,平安银行,-57323000000,61382000000,19021000000,平安银行
1,000001,2019-01-03,9.18,9.33,9.15,9.28,0.009793,平安银行,1991-04-03,10,...,36540000000,NaN,NaN,NaN,24818000000,平安银行,-57323000000,61382000000,19021000000,平安银行
2,000001,2019-01-04,9.24,9.82,9.22,9.75,0.050647,平安银行,1991-04-03,10,...,36540000000,NaN,NaN,NaN,24818000000,平安银行,-57323000000,61382000000,19021000000,平安银行
3,000001,2019-01-07,9.84,9.85,9.63,9.74,-0.001026,平安银行,1991-04-03,10,...,36540000000,NaN,NaN,NaN,24818000000,平安银行,-57323000000,61382000000,19021000000,平安银行
4,000001,2019-01-08,9.73,9.74,9.62,9.66,-0.008214,平安银行,1991-04-03,10,...,36540000000,NaN,NaN,NaN,24818000000,平安银行,-57323000000,61382000000,19021000000,平安银行
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5000407,900957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,81882874.95,NaN,7907748.5,38843715.6,30015892.98,凌云B股,133876680.38,-10129431,-83419306.44,凌云B股
5000408,900957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,63733263.62,NaN,5426585.84,28900349.1,18224007.47,凌云B股,17675525.34,-21115.35,-31465040.73,凌云B股
5000409,900957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,42807516.31,NaN,3942872.92,18888703.28,11187154.83,凌云B股,8837480.07,-9688,-20944609.18,凌云B股
5000410,900957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20546598.07,NaN,1442184.13,9134941.6,5324024.45,凌云B股,-3067349.05,-500000,-10836179.78,凌云B股


In [11]:
###todo：要删除的重复列名
deleteColumns=["Trddt_quarter","FS_Combas.Typrep","FS_Combas.ShortName","csmar_listedcoinfo.Stknme","csmar_listedcoinfo.Conme","FS_Comscfd.ShortName","FS_Comins.ShortName","csmar_listedcoinfo.Conme_en","csmar_listedcoinfo.Nnindcd","csmar_listedcoinfo.PROVINCECD","csmar_listedcoinfo.CITY","csmar_listedcoinfo.Nnindnme"]
afterDeleteData=allData[[i for i in allData.columns if i not in deleteColumns]]
afterDeleteData

,Stkcd,Trddt,Opnprc,Hiprc,Loprc,Clsprc,ChangeRatio,Stknme,Listdt,Cuntrycd,...,csmar_listedcoinfo.BUSSINESSRANGE,FS_Comins.B001100000,FS_Comins.B001200000,FS_Comins.B001209000,FS_Comins.B001210000,FS_Comins.B001211000,FS_Comins.B002000000,FS_Comscfd.C001000000,FS_Comscfd.C002000000,FS_Comscfd.C003000000
0,000001,2019-01-02,9.39,9.42,9.16,9.19,-0.020256,平安银行,1991-04-03,10,...,办理人民币存、贷、结算、汇兑业务；人民币票据承兑和贴现；各项信托业务；经监管机构批准发行或买...,106212000000,36540000000,NaN,NaN,NaN,24818000000,-57323000000,61382000000,19021000000
1,000001,2019-01-03,9.18,9.33,9.15,9.28,0.009793,平安银行,1991-04-03,10,...,办理人民币存、贷、结算、汇兑业务；人民币票据承兑和贴现；各项信托业务；经监管机构批准发行或买...,106212000000,36540000000,NaN,NaN,NaN,24818000000,-57323000000,61382000000,19021000000
2,000001,2019-01-04,9.24,9.82,9.22,9.75,0.050647,平安银行,1991-04-03,10,...,办理人民币存、贷、结算、汇兑业务；人民币票据承兑和贴现；各项信托业务；经监管机构批准发行或买...,106212000000,36540000000,NaN,NaN,NaN,24818000000,-57323000000,61382000000,19021000000
3,000001,2019-01-07,9.84,9.85,9.63,9.74,-0.001026,平安银行,1991-04-03,10,...,办理人民币存、贷、结算、汇兑业务；人民币票据承兑和贴现；各项信托业务；经监管机构批准发行或买...,106212000000,36540000000,NaN,NaN,NaN,24818000000,-57323000000,61382000000,19021000000
4,000001,2019-01-08,9.73,9.74,9.62,9.66,-0.008214,平安银行,1991-04-03,10,...,办理人民币存、贷、结算、汇兑业务；人民币票据承兑和贴现；各项信托业务；经监管机构批准发行或买...,106212000000,36540000000,NaN,NaN,NaN,24818000000,-57323000000,61382000000,19021000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5000407,900957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,旅游景点开发、旅游基础设施的建设、综合开发业务；房地产开发及商品房销售、物业管理；相关的咨询...,98113165.84,81882874.95,NaN,7907748.5,38843715.6,30015892.98,133876680.38,-10129431,-83419306.44
5000408,900957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,旅游景点开发、旅游基础设施的建设、综合开发业务；房地产开发及商品房销售、物业管理；相关的咨询...,75804279.65,63733263.62,NaN,5426585.84,28900349.1,18224007.47,17675525.34,-21115.35,-31465040.73
5000409,900957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,旅游景点开发、旅游基础设施的建设、综合开发业务；房地产开发及商品房销售、物业管理；相关的咨询...,49636293.55,42807516.31,NaN,3942872.92,18888703.28,11187154.83,8837480.07,-9688,-20944609.18
5000410,900957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,旅游景点开发、旅游基础设施的建设、综合开发业务；房地产开发及商品房销售、物业管理；相关的咨询...,22751337.48,20546598.07,NaN,1442184.13,9134941.6,5324024.45,-3067349.05,-500000,-10836179.78


In [12]:
### 检查是否有缺失值达到50%的，有的话就删除
totalNumRows=5000412
num = afterDeleteData.isna().sum()
indexs=[]
nullNums=[]
nullPerc=[]
for index in num.index:
    indexs.append(index)
    nullNums.append(num[index])
    nullPerc.append(str(round(float(num[index]/totalNumRows),3)*100)+" %")

num=pd.DataFrame.from_dict({"Variance":indexs,"Number Of Null":nullNums,"Percentage Of Null":nullPerc})
num

,Variance,Number Of Null,Percentage Of Null
0,Stkcd,0,0.0 %
1,Trddt,32438,0.6 %
2,Opnprc,32438,0.6 %
3,Hiprc,32438,0.6 %
4,Loprc,32438,0.6 %
5,Clsprc,32438,0.6 %
6,ChangeRatio,32438,0.6 %
7,Stknme,32438,0.6 %
8,Listdt,32438,0.6 %
9,Cuntrycd,32438,0.6 %


In [13]:
###todo：删除和处理回归不能输入的数据,删除回报率为null的行,同时删除缺失值过多的 [FS_Combas.A001127000,FS_Combas.A001220000,FS_Combas.A001219000,FS_Combas.A001109000] 列
uselessColumns=["FS_Combas.A001127000","FS_Combas.A001220000","FS_Combas.A001219000","FS_Combas.A001109000","Stknme","Listdt","Trddt","csmar_listedcoinfo.BUSSINESSRANGE","Conme_en","PROVINCECODE","Parvcur","csmar_listedcoinfo.MAINBUSSINESS","CITYCODE","PROVINCE","Cuntrycd","Conme","Nindcd","Nindcd","Nnindcd","OWNERSHIPTYPECODE"]

uselessDeletedDf=allData[[i for i in allData.columns if i not in deleteColumns and i not in uselessColumns]]
uselessDeletedDf=uselessDeletedDf.dropna(how="all")
uselessDeletedDf

,Stkcd,Opnprc,Hiprc,Loprc,Clsprc,ChangeRatio,Indcd,Indnme,Nindnme,Nnindnme,...,csmar_listedcoinfo.OWNERSHIPTYPE,FS_Comins.B001100000,FS_Comins.B001200000,FS_Comins.B001209000,FS_Comins.B001210000,FS_Comins.B001211000,FS_Comins.B002000000,FS_Comscfd.C001000000,FS_Comscfd.C002000000,FS_Comscfd.C003000000
0,000001,9.39,9.42,9.16,9.19,-0.020256,0001,金融,银行业,货币金融服务,...,私营企业,106212000000,36540000000,NaN,NaN,NaN,24818000000,-57323000000,61382000000,19021000000
1,000001,9.18,9.33,9.15,9.28,0.009793,0001,金融,银行业,货币金融服务,...,私营企业,106212000000,36540000000,NaN,NaN,NaN,24818000000,-57323000000,61382000000,19021000000
2,000001,9.24,9.82,9.22,9.75,0.050647,0001,金融,银行业,货币金融服务,...,私营企业,106212000000,36540000000,NaN,NaN,NaN,24818000000,-57323000000,61382000000,19021000000
3,000001,9.84,9.85,9.63,9.74,-0.001026,0001,金融,银行业,货币金融服务,...,私营企业,106212000000,36540000000,NaN,NaN,NaN,24818000000,-57323000000,61382000000,19021000000
4,000001,9.73,9.74,9.62,9.66,-0.008214,0001,金融,银行业,货币金融服务,...,私营企业,106212000000,36540000000,NaN,NaN,NaN,24818000000,-57323000000,61382000000,19021000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5000407,900957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,私营企业,98113165.84,81882874.95,NaN,7907748.5,38843715.6,30015892.98,133876680.38,-10129431,-83419306.44
5000408,900957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,私营企业,75804279.65,63733263.62,NaN,5426585.84,28900349.1,18224007.47,17675525.34,-21115.35,-31465040.73
5000409,900957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,私营企业,49636293.55,42807516.31,NaN,3942872.92,18888703.28,11187154.83,8837480.07,-9688,-20944609.18
5000410,900957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,私营企业,22751337.48,20546598.07,NaN,1442184.13,9134941.6,5324024.45,-3067349.05,-500000,-10836179.78


In [14]:
# afterDeleteData.to_csv("stockData/furtherInformation/afterDeleteData.csv")